In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load the data (assuming it's in a CSV-like format)
data = pd.read_csv('/Users/nilaysinghsolanki/Downloads/industry_excess_returns.csv', parse_dates=['Date'], index_col='Date')

# Convert to numpy array for calculations (540 months x 17 industries)
returns = data.values

# Calculate expected monthly returns (mean) and covariance matrix
mean_returns = np.mean(returns, axis=0)
cov_matrix = np.cov(returns, rowvar=False)

# Number of assets
n_assets = len(mean_returns)

In [4]:
# Function to calculate portfolio return and volatility
def portfolio_performance(weights, mean_returns, cov_matrix):
    port_return = np.sum(weights * mean_returns)
    port_volatility = np.sqrt(weights.T @ cov_matrix @ weights)
    return port_return, port_volatility

# Simulate random portfolios
n_portfolios = 10000
results = np.zeros((3, n_portfolios))  # Store return, volatility, Sharpe ratio
weights_record = []

for i in range(n_portfolios):
    weights = np.random.random(n_assets)
    weights /= np.sum(weights)  # Normalize to sum to 1
    weights_record.append(weights)
    
    port_return, port_volatility = portfolio_performance(weights, mean_returns, cov_matrix)
    results[0, i] = port_return  # Monthly return
    results[1, i] = port_volatility  # Monthly volatility
    results[2, i] = (port_return - 0.00329) / port_volatility  # Sharpe ratio with Rf = 0.329%

# Extract efficient frontier (portfolios with max return for given volatility)
volatility_range = np.linspace(min(results[1]), max(results[1]), 100)
efficient_returns = []

for vol in volatility_range:
    idx = np.where(results[1] <= vol)[0]
    if len(idx) > 0:
        max_return = max(results[0, idx])
        efficient_returns.append(max_return)
    else:
        efficient_returns.append(np.nan)

In [5]:
from scipy.optimize import minimize

# Objective: Negative Sharpe Ratio (to maximize)
def neg_sharpe_ratio(weights, mean_returns, cov_matrix, rf):
    port_return, port_volatility = portfolio_performance(weights, mean_returns, cov_matrix)
    return -(port_return - rf) / port_volatility

# Constraints: Weights sum to 1
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
# Bounds: No short-selling (0 to 1 for each weight)
bounds = tuple((0, 1) for _ in range(n_assets))
# Initial guess: Equal weights
init_weights = np.array([1/n_assets] * n_assets)

# Optimize
rf = 0.00329  # Monthly risk-free rate
opt_result = minimize(neg_sharpe_ratio, init_weights, args=(mean_returns, cov_matrix, rf),
                      method='SLSQP', bounds=bounds, constraints=constraints)

tangency_weights = opt_result.x
tangency_return, tangency_volatility = portfolio_performance(tangency_weights, mean_returns, cov_matrix)
tangency_sharpe = (tangency_return - rf) / tangency_volatility

print(f"Tangency Portfolio Return (Monthly): {tangency_return:.4%}")
print(f"Tangency Portfolio Volatility (Monthly): {tangency_volatility:.4%}")
print(f"Tangency Portfolio Sharpe Ratio: {tangency_sharpe:.4f}")
print("Tangency Portfolio Weights:", {data.columns[i]: f"{w:.4f}" for i, w in enumerate(tangency_weights) if w > 0})

Tangency Portfolio Return (Monthly): 0.8498%
Tangency Portfolio Volatility (Monthly): 3.9041%
Tangency Portfolio Sharpe Ratio: 0.1334
Tangency Portfolio Weights: {'Food ': '0.1713', 'Clths': '0.0000', 'Durbl': '0.0000', 'Cnsum': '0.4270', 'Steel': '0.0000', 'Cars ': '0.0369', 'Utils': '0.0796', 'Rtail': '0.2852'}


Funds

In [40]:
import numpy as np
import pandas as pd

# Load mutual fund returns
fund_data = pd.read_csv('/Users/nilaysinghsolanki/Downloads/MutualFunds_updated.csv', index_col='Date')
fund_data.index = pd.to_datetime(fund_data.index, format='%d/%m/%y')
fund_returns = fund_data[['MITTX', 'FBGRX', 'FSCSX']].values  # 276 x 3 (Jan 2000 - Nov 2023)

# Load market portfolio (Mkt-RF)
mkt_data = pd.read_csv('/Users/nilaysinghsolanki/Downloads/Market_Portfolio.csv', index_col='Date')
mkt_data.index = pd.to_datetime(mkt_data.index, format='%d/%m/%y')

# Sort the data by date to avoid non-monotonic error
mkt_data.sort_index(inplace=True)

# Check the range of dates in the market data
print(f"Date range in market data: {mkt_data.index.min()} to {mkt_data.index.max()}")

# Ensure the date range is correct and no missing values
mkt_data = mkt_data.dropna()  # Drop any rows with missing data
mkt_data = mkt_data.loc['2000-01-31':'2023-11-30']  # Filter to match fund data period

# Verify the index is sorted correctly
assert mkt_data.index.is_monotonic_increasing, "Dates in market data are not sorted correctly."

mkt_rf = mkt_data['Mkt-RF'].values / 100  # Convert percentage to decimal

# Use fixed risk-free rate of 0.329%
rf = 0.329 / 100  # Convert percentage to decimal



# Calculate metrics for each fund
fund_names = ['MITTX', 'FBGRX', 'FSCSX']
for i, fund_name in enumerate(fund_names):
    fund_ret = fund_returns[:, i]
    mean_fund = np.mean(fund_ret)
    std_fund = np.std(fund_ret)
    
    # Beta
    cov_fund_mkt = np.cov(fund_ret, mkt_rf)[0, 1]
    var_mkt_rf = np.var(mkt_rf)
    beta = cov_fund_mkt / var_mkt_rf
    
    # Sharpe Ratio
    sharpe_ratio = (mean_fund - rf) / std_fund
    
    # Treynor Ratio
    treynor_ratio = (mean_fund - rf) / beta
    
    # Jensen’s Alpha
    mean_mkt_rf = np.mean(mkt_rf)
    jensen_alpha = mean_fund - (rf + beta * mean_mkt_rf)
    
    # Print results
    print(f"\nMetrics for {fund_name}:")
    print(f"Mean Return (Monthly): {mean_fund:.4%}")
    print(f"Volatility (Monthly): {std_fund:.4%}")
    print(f"Beta: {beta:.4f}")
    print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
    print(f"Treynor Ratio: {treynor_ratio:.4%}")
    print(f"Jensen’s Alpha: {jensen_alpha:.4%}")

# Additional diagnostics
print(f"\nRisk-Free Rate (Monthly): {rf:.4%}")
print(f"Mean Mkt-RF (Monthly): {np.mean(mkt_rf):.4%}")


Date range in market data: 2000-01-31 00:00:00 to 2023-12-31 00:00:00

Metrics for MITTX:
Mean Return (Monthly): 0.2889%
Volatility (Monthly): 4.4499%
Beta: 0.0115
Sharpe Ratio: -0.0090
Treynor Ratio: -3.4882%
Jensen’s Alpha: -0.0463%

Metrics for FBGRX:
Mean Return (Monthly): 0.5330%
Volatility (Monthly): 5.4600%
Beta: 0.0370
Sharpe Ratio: 0.0374
Treynor Ratio: 5.5115%
Jensen’s Alpha: 0.1841%

Metrics for FSCSX:
Mean Return (Monthly): 0.6762%
Volatility (Monthly): 7.3643%
Beta: 0.0330
Sharpe Ratio: 0.0472
Treynor Ratio: 10.5343%
Jensen’s Alpha: 0.3296%

Risk-Free Rate (Monthly): 0.3290%
Mean Mkt-RF (Monthly): 0.5362%


In [48]:
# Given Values
tangency_return = 0.008498  # Tangency Portfolio Return (Monthly)
tangency_volatility = 0.039041  # Tangency Portfolio Volatility (Monthly)
sharpe_ratio_tangency = 0.1334  # Tangency Portfolio Sharpe Ratio
risk_free_rate = 0.00329  # Monthly Risk-Free Rate (0.329%)
market_return = 0.005362  # Monthly Market Risk Premium (Mkt-RF)
market_volatility = 0.04  # Assuming a typical market volatility of 4% (0.04)

# 1. **Beta (Tangency Portfolio)**
beta_tangency = (sharpe_ratio_tangency * tangency_volatility) / market_volatility

# 2. **Treynor Ratio for Tangency Portfolio**
treynor_ratio_tangency = (tangency_return - risk_free_rate) / beta_tangency

# 3. **Jensen’s Alpha for Tangency Portfolio**
jensens_alpha_tangency = tangency_return - (risk_free_rate + beta_tangency * (market_return - risk_free_rate))

# Output the computed metrics
print(f"Tangency Portfolio Beta: {beta_tangency:.4f}")
print(f"Tangency Portfolio Treynor Ratio: {treynor_ratio_tangency:.4f}")
print(f"Tangency Portfolio Jensen's Alpha: {jensens_alpha_tangency:.4f}")


Tangency Portfolio Beta: 0.1302
Tangency Portfolio Treynor Ratio: 0.0400
Tangency Portfolio Jensen's Alpha: 0.0049
